In [16]:
import os
import urllib
import psycopg2
import subprocess
import numpy as np
import pandas as pd
from tqdm import tqdm
from numpy import NaN
from datetime import datetime  
from keras.models import load_model
from collections import OrderedDict
from sqlalchemy import create_engine

In [17]:
db_setting = pd.read_csv(os.getcwd()+'\\'+'set_file'+'\\'+'dbsetting.csv',encoding='ANSI') #database設定檔
time_setting = pd.read_csv(f'{os.getcwd()}'+'\\'+'set_file'+'\\'+'time.csv',encoding='ANSI') #讀取時間設定檔
hydro_setting = pd.read_csv(f'{os.getcwd()}'+'\\'+'set_file'+'\\'+'hydro_setting.csv',encoding='ANSI') #讀取氣象水文設定檔
farm = pd.read_csv(f'{os.getcwd()}'+'\\'+'tem'+'\\'+'farm_v3.csv',encoding='ANSI') #讀取坵塊資訊

In [18]:
#將坵塊資訊轉英文
farm.rename(columns={'處代碼':'placeID','工作站代碼':'workID','坵塊編號':'farmID','海拔高度(m)':'altitude','競用區':'conID','經度':'long','緯度':'lat','絕對最高氣溫(℃)':'tx01_max','絕對最低氣溫(℃)':'tx01_min','平均風速(m/s)':'wd01','相對溼度(%)':'rh01','平均日照時數':'ss01','累積降水量(mm)':'pp01','土壤質地':'soil','作物面積(平方公尺)':'area'},inplace=True)
farm['conID'] = farm['conID'].map(lambda x:'Liyutan' if x =='鯉魚潭' else('Pao-Shan' if x=='寶山' else ('Mingde' if x=='明德' else('Ishioka' if x=='石岡壩' else('Shimen' if x=='石門' else x)))))
farm.drop('Unnamed: 0',axis=1,inplace=True)
#處理坵塊缺失/變數轉換
farm['soil'] = farm['soil'].replace('-',np.NaN)
farm[['placeID','workID','soil']] = farm[['placeID','workID','soil']].fillna(-9999)
farm[['placeID','workID','soil','altitude']] = farm[['placeID','workID','soil','altitude']].astype('int')

In [4]:
#connect db1
database1=db_setting['db1'][0]
user1 =db_setting['db1'][1]
passwords1=db_setting['db1'][2]
host1=db_setting['db1'][3]
port1=db_setting['db1'][4]
gssencmode1=db_setting['db1'][5]
engine1 = create_engine(f'postgresql+psycopg2://{user1}:%s@{host1}/{database1}?gssencmode={gssencmode1}'%urllib.parse.quote_plus(passwords1))
conn = psycopg2.connect(database=database1,
                        user=user1,
                        password=passwords1,
                        host=host1,
                        port=port1,
                        gssencmode=gssencmode1)
                        
#connect db2
database2=db_setting['db2'][0]
user2 =db_setting['db2'][1]
passwords2=db_setting['db2'][2]
host2=db_setting['db2'][3]
port2=db_setting['db2'][4]
gssencmode2=db_setting['db2'][5]
engine2 = create_engine(f'postgresql+psycopg2://{user2}:%s@{host2}/{database2}?gssencmode={gssencmode2}'%urllib.parse.quote_plus(passwords2))
conn = psycopg2.connect(database=database2,
                        user=user2,
                        password=passwords2,
                        host=host2,
                        port=port2,
                        gssencmode=gssencmode2)
                        

In [5]:
auto_column_list = ", ".join((hydro_setting['Input'][0],hydro_setting['Input'][2],hydro_setting['Input'][3]))
cwb_agr_column_list = ", ".join((hydro_setting['Input'][0],hydro_setting['Input'][1],hydro_setting['Input'][2],hydro_setting['Input'][3]))
qpesums_column_list = (hydro_setting['Input'][4])
qpesums_database_table = 'QPESUMS_history_grid_day'
cwb_database_table = 'cwb_history_hour'
agr_database_table = 'cwbagr_weather_obs_hr'
auto_database_table = 'cwb_history_auto_hour'
start_time = pd.to_datetime(str(time_setting['Input'][0]) + '_' + str(time_setting['Input'][1]) + '_' + str(time_setting['Input'][2]),format='%Y_%m_%d')
end_time = pd.to_datetime(str(time_setting['Input'][3]) + '_' + str(time_setting['Input'][4]) + '_' + str(time_setting['Input'][5]),format='%Y_%m_%d')
end_time = end_time.replace(hour=23, minute=0)
start_time_year = int(start_time.year)
end_time_year = int(end_time.year)

In [6]:
#build up the dic for Info(hour to day) to carry further calculation
day_input_methods ={}
for Input,Info in zip(hydro_setting['Input'],hydro_setting['Info(hour to day)']): 
    day_input_methods[Input] = Info

In [7]:
#build up the dic for Info(day to ten_days) to carry further calculation
ten_days_input_methods ={}
for Input,Info in zip(hydro_setting['Input'],hydro_setting['Info(day to ten_days)']): 
    ten_days_input_methods[Input] = Info

In [8]:
#hour to day calculation
def day_cal_variable(var_name: str, calculate_method: str) -> pd.DataFrame:
    if var_name in day_input_methods:
        if str(day_input_methods[var_name]) == '5':
            all_df[var_name] = df.groupby('stno').resample('D').sum()[var_name].values
        elif str(day_input_methods[var_name]) == '4':
            all_df[var_name] = df.groupby('stno').resample('D').mean()[var_name].values
        elif str(day_input_methods[var_name]) == '3':
            all_df[var_name+"_max"] = df.groupby('stno').resample('D').max()[var_name].values
        elif str(day_input_methods[var_name]) == '2':
            all_df[var_name+"_min"] = df.groupby('stno').resample('D').min()[var_name].values
        elif str(day_input_methods[var_name]) == '1':
            all_df[var_name+"_min"] = df.groupby('stno').resample('D').min()[var_name].values
            all_df[var_name+"_max"] = df.groupby('stno').resample('D').max()[var_name].values
            
#ten_days calculation
def ten_days_cal(df:pd.DataFrame(),var:str)->pd.DataFrame():
    if var =='tx01_min':
        df[var] = round(all_dff.groupby(['year','month','ten_days','stno'])[var].min(),2)

    elif var =='tx01_max':
        df[var] = round(all_dff.groupby(['year','month','ten_days','stno'])[var].max(),2)
    else: 
        if ten_days_input_methods[var] == 5:
            df[var] = round(all_dff.groupby(['year','month','ten_days','stno'])[var].sum(),2)
        elif ten_days_input_methods[var] == 4:
            df[var] = round(all_dff.groupby(['year','month','ten_days','stno'])[var].mean(),2)
        elif ten_days_input_methods[var] == 3:
            df[var] = round(all_dff.groupby(['year','month','ten_days','stno'])[var].max(),2)
        elif ten_days_input_methods[var] == 2:
            df[var] = round(all_dff.groupby(['year','month','ten_days','stno'])[var].min(),2)
        elif ten_days_input_methods[var] == 1:
            df[var+"_min"] = round(all_dff.groupby(['year','month','ten_days','stno'])[var].min(),2)
            df[var+"_max"] = round(all_dff.groupby(['year','month','ten_days','stno'])[var].max(),2)
           

In [9]:
#build up the value dataset for each farm ID.

def data_filledIn(tem_df, index_num, stno, x: str) -> pd.DataFrame:
    if isinstance(x,str):
        if x[0] =='C':
            if index_num == 0:
                tem_df['tx01_max'] = auto_final_df[auto_final_df['stno'] == stno]['tx01_max'].reset_index(drop=True)
            elif index_num == 1:
                tem_df['tx01_min'] = auto_final_df.loc[auto_final_df['stno'] == stno, 'tx01_min'].reset_index(drop=True)
            elif index_num == 2:
                tem_df['wd01'] = auto_final_df.loc[auto_final_df['stno'] == stno, 'wd01'].reset_index(drop=True)
            else:
                tem_df['rh01'] = auto_final_df.loc[auto_final_df['stno'] == stno, 'rh01'].reset_index(drop=True)
        elif x[0] == '4':
            if index_num == 0:
                tem_df['tx01_max'] = cwb_final_df.loc[cwb_final_df['stno'] == stno, 'tx01_max'].reset_index(drop=True)
            elif index_num == 1:
                tem_df['tx01_min'] = cwb_final_df.loc[cwb_final_df['stno'] == stno, 'tx01_min'].reset_index(drop=True)
            elif index_num == 2:
                tem_df['wd01'] = cwb_final_df.loc[cwb_final_df['stno'] == stno, 'wd01'].reset_index(drop=True)
            elif index_num == 3:
                tem_df['rh01'] = cwb_final_df.loc[cwb_final_df['stno'] == stno, 'rh01'].reset_index(drop=True)
            else:
                tem_df['ss01'] = cwb_final_df.loc[cwb_final_df['stno'] == stno, 'ss01'].reset_index(drop=True)
        else:
            if index_num == 0:
                tem_df['tx01_max'] = agr_final_df.loc[agr_final_df['stno'] == stno, 'tx01_max'].reset_index(drop=True)
            elif index_num == 1:
                tem_df['tx01_min'] = agr_final_df.loc[agr_final_df['stno'] == stno, 'tx01_min'].reset_index(drop=True)
            elif index_num == 2:
                tem_df['wd01'] = agr_final_df.loc[agr_final_df['stno'] == stno, 'wd01'].reset_index(drop=True)
            elif index_num == 3:
                tem_df['rh01'] = agr_final_df.loc[agr_final_df['stno'] == stno, 'rh01'].reset_index(drop=True)
            else:
                tem_df['ss01'] = agr_final_df.loc[agr_final_df['stno'] == stno, 'ss01'].reset_index(drop=True)
    elif isinstance(x,np.int64):
        tem_df['pp01'] = qpesums_final_df.loc[qpesums_final_df['stno'] == stno, 'pp01'].reset_index(drop=True)
    
    tem_df['year'] = cwb_final_df[cwb_final_df['stno'] == '467050']['year'].reset_index(drop=True)
    tem_df['month'] = cwb_final_df[cwb_final_df['stno'] == '467050']['month'].reset_index(drop=True)
    tem_df['ten_days'] = cwb_final_df[cwb_final_df['stno'] == '467050']['ten_days'].reset_index(drop=True)
    tem_df.index = range(len(tem_df))

    return tem_df


In [10]:
# Get all the value we need from all the farmID. We stored them in each list for further retrieve purpose when build up data_filledIn this mentioned above.

auto_db =[]
cwb_db =[]
agr_db=[]
qpesums_db =[]

for i in set(farm['tx01_max']):
    if isinstance(i,str):
        if i[0] =='C':
            auto_db.append(i)    
        elif i[0] == '4':
            cwb_db.append(i)
        else :
            agr_db.append(i)
    else:
        qpesums_db.append(i)

for i in set(farm['ss01']):
    if isinstance(i,str):
        if i[0] =='C':
            auto_db.append(i)    
        elif i[0] == '4':
            cwb_db.append(i)
        else :
            agr_db.append(i)
    else:
        qpesums_db.append(i)

for i in set(farm['pp01']):
    if isinstance(i,str):
        if i[0] =='C':
            auto_db.append(i)    
        elif i[0] == '4':
            cwb_db.append(i)
        else :
            agr_db.append(i)
    else:
        qpesums_db.append(i)
auto_db = tuple(set(auto_db))
cwb_db =tuple(set(cwb_db))
agr_db=tuple(set(agr_db))
qpesums_db =tuple(set(qpesums_db))

#auto_db
query = f"select obsdate, obshour,stno, {auto_column_list} from {auto_database_table}_{start_time.year} where obsdate between '{start_time}' and '{end_time}' and (stno in {auto_db})"
df = pd.read_sql(query,con=engine1)
df['obstime'] = pd.to_datetime(df['obsdate'].astype(str)+'_'+df['obshour'].astype(str),format='%Y-%m-%d_%H')
df[df.isin([-9999,-9997,-9996,-9991])] = np.nan
df.set_index(df['obstime'],inplace=True)
all_df = pd.DataFrame()
auto_dict = {key: value for key, value in day_input_methods.items() if (key == hydro_setting['Input'][0]) | (key == hydro_setting['Input'][2]) | (key == hydro_setting['Input'][3])}
for j in auto_dict:
    day_cal_variable(f'{j}', f"{day_input_methods[j]}")
# add obstime and stno columns
all_df['obstime'] = df.groupby('stno').resample('D')['obstime'].first().values
all_df['stno'] = df.groupby('stno').resample('D')['stno'].first().values
all_dff = all_df.copy()
all_dff['obstime'] = pd.to_datetime(all_dff['obstime'], infer_datetime_format=True)
all_dff['year'] = all_dff['obstime'].dt.year
all_dff['month'] = all_dff['obstime'].dt.month
all_dff['day'] = all_dff['obstime'].dt.day
all_dff['ten_days'] = all_dff['day'].apply(lambda x:'1' if x <= 10 else ('2' if x <=20 else '3'))
all_dff.reset_index(drop=True,inplace=True)
all_dff_auto_hydro_col = all_dff.drop(['obstime', 'stno', 'year', 'month', 'day', 'ten_days'],axis=1)
all_dff_auto_hydro_col = list(all_dff_auto_hydro_col.columns)
all_dff2 = pd.DataFrame()
for i in all_dff_auto_hydro_col:
    ten_days_cal(all_dff2,i)
all_dff2.fillna(-9999,inplace=True)
all_dff2.reset_index(inplace=True)
auto_final_df = all_dff2.copy()


#cwb_db
query = f"select obsdate, obshour,stno, {cwb_agr_column_list} from {cwb_database_table}_{start_time.year} where obsdate between '{start_time}' and '{end_time}' and (stno in {cwb_db})"
df = pd.read_sql(query,con=engine1)
df['obstime'] = pd.to_datetime(df['obsdate'].astype(str)+'_'+df['obshour'].astype(str),format='%Y-%m-%d_%H')
df[df.isin([-9999,-9997,-9996,-9991,-9998])] = np.nan
df.set_index(df['obstime'],inplace=True)
all_df = pd.DataFrame()
cwb_dict = {key: value for key, value in day_input_methods.items() if (key == hydro_setting['Input'][0]) | (key == hydro_setting['Input'][1]) |(key == hydro_setting['Input'][2]) | (key == hydro_setting['Input'][3])}
for j in cwb_dict:
    day_cal_variable(f'{j}', f"{day_input_methods[j]}")
# add obstime and stno columns
all_df['obstime'] = df.groupby('stno').resample('D')['obstime'].first().values
all_df['stno'] = df.groupby('stno').resample('D')['stno'].first().values
all_dff = all_df.copy()
all_dff['obstime'] = pd.to_datetime(all_dff['obstime'], infer_datetime_format=True)
all_dff['year'] = all_dff['obstime'].dt.year
all_dff['month'] = all_dff['obstime'].dt.month
all_dff['day'] = all_dff['obstime'].dt.day
all_dff['ten_days'] = all_dff['day'].apply(lambda x:'1' if x <= 10 else ('2' if x <=20 else '3'))
all_dff.reset_index(drop=True,inplace=True)
all_dff_cwb_hydro_col = all_dff.drop(['obstime', 'stno', 'year', 'month', 'day', 'ten_days'],axis=1)
all_dff_cwb_hydro_col = list(all_dff_cwb_hydro_col.columns)
all_dff2 = pd.DataFrame()
for i in all_dff_cwb_hydro_col:
    ten_days_cal(all_dff2,i)
all_dff2.fillna(-9999,inplace=True)
all_dff2.reset_index(inplace=True)
cwb_final_df = all_dff2.copy()

#agr_db
query = f"select obstime,station, {cwb_agr_column_list} from {agr_database_table}_{start_time.year}_station where obstime between '{start_time}' and '{end_time}' and (station in {agr_db})"
df = pd.read_sql(query,con=engine2)
df.rename(columns={'station':'stno'},inplace=True)
df['obstime'] = pd.to_datetime(df['obstime'],format='%Y-%m-%d')
df['obshour'] = df['obstime'].dt.hour
df[df.isin([-9999,-9997,-9996,-9991,-9998])] = np.nan
df.set_index(df['obstime'],inplace=True)
all_df = pd.DataFrame()
agr_dict = {key: value for key, value in day_input_methods.items() if (key == hydro_setting['Input'][0]) | (key == hydro_setting['Input'][1]) |(key == hydro_setting['Input'][2]) | (key == hydro_setting['Input'][3])}
for j in agr_dict:
    day_cal_variable(f'{j}', f"{day_input_methods[j]}")
# add obstime and stno columns
all_df['obstime'] = df.groupby('stno').resample('D')['obstime'].first().values
all_df['stno'] = df.groupby('stno').resample('D')['stno'].first().values
all_dff = all_df.copy()
all_dff['obstime'] = pd.to_datetime(all_dff['obstime'], infer_datetime_format=True)
all_dff['year'] = all_dff['obstime'].dt.year
all_dff['month'] = all_dff['obstime'].dt.month
all_dff['day'] = all_dff['obstime'].dt.day
all_dff['ten_days'] = all_dff['day'].apply(lambda x:'1' if x <= 10 else ('2' if x <=20 else '3'))
all_dff.reset_index(drop=True,inplace=True)
all_dff_agr_hydro_col = all_dff.drop(['obstime', 'stno', 'year', 'month', 'day', 'ten_days'],axis=1)
all_dff_agr_hydro_col = list(all_dff_agr_hydro_col.columns)
all_dff2 = pd.DataFrame()
for i in all_dff_agr_hydro_col:
    ten_days_cal(all_dff2,i)
all_dff2.fillna(-9999,inplace=True)
all_dff2.reset_index(inplace=True)
agr_final_df = all_dff2.copy()

#qpesums_db
start_time_str = start_time.strftime("%Y-%m-%d")
end_time_str = end_time.strftime("%Y-%m-%d")
query = f'select grid,value,date from "{qpesums_database_table}_{start_time.year}_37134"' + f" where date between '{start_time_str}' and '{end_time_str}' and (grid in {qpesums_db})"
df = pd.read_sql(query,con=engine2)
df.rename(columns={'grid':'stno','value':'pp01'},inplace=True)
df['obstime'] = pd.to_datetime(df['date'].astype(str),format='%Y-%m-%d')
df.drop('date',axis=1,inplace=True)
df[df.isin([-9999,-9997,-9996,-9991,-9998])] = np.nan
all_dff = df.copy()
all_dff['year'] = all_dff['obstime'].dt.year
all_dff['month'] = all_dff['obstime'].dt.month
all_dff['day'] = all_dff['obstime'].dt.day
all_dff['ten_days'] = all_dff['day'].apply(lambda x:'1' if x <= 10 else ('2' if x <=20 else '3'))
all_dff.reset_index(drop=True,inplace=True)
all_dff_qpesums_hydro_col = all_dff.drop(['obstime', 'stno', 'year', 'month', 'day', 'ten_days'],axis=1)
all_dff_qpesums_hydro_col = list(all_dff_qpesums_hydro_col.columns)
all_dff2 = pd.DataFrame()
for i in all_dff_qpesums_hydro_col:
    ten_days_cal(all_dff2,i)
all_dff2.fillna(-9999,inplace=True)
all_dff2.reset_index(inplace=True)
qpesums_final_df = all_dff2.copy()

In [ ]:
#test
final_DF = pd.DataFrame()
for i in tqdm(range(len(farm.index))):
    tem_df = pd.DataFrame()
    placeID = farm.iloc[i,:]['placeID']
    workID = farm.iloc[i,:]['workID']
    farmID = farm.iloc[i,:]['farmID']
    long = farm.iloc[i,:]['long']
    lat = farm.iloc[i,:]['lat']
    altitude = farm.iloc[i,:]['altitude']
    conID = farm.iloc[i,:]['conID']
    soil = farm.iloc[i,:]['soil']
    area = farm.iloc[i,:]['area']
    tx01_max = farm.iloc[i,:]['tx01_max']
    tx01_min = farm.iloc[i,:]['tx01_min']
    wd01 = farm.iloc[i,:]['wd01']
    rh01 = farm.iloc[i,:]['rh01']
    ss01 = farm.iloc[i,:]['ss01']
    pp01 = farm.iloc[i,:]['pp01']
    hydro_lst = [tx01_max, tx01_min, wd01, rh01, ss01, pp01]

    for index, hydro_code in enumerate(hydro_lst):
        tem_df = data_filledIn(tem_df, index, hydro_code, hydro_code)
    
    final_DF = pd.concat([final_DF, tem_df])
    final_DF['placeID'] = placeID
    final_DF['workID'] = workID
    final_DF['farmID'] = farmID
    final_DF['long'] = long
    final_DF['lat'] = lat
    final_DF['altitude'] = altitude
    final_DF['conID'] = conID
    final_DF['soil'] = soil
    final_DF['area'] = area
    print(final_DF)
    break

In [ ]:
# Since farmID may vary each time we retrieve its ID, we will take its index to make sure we don't miss any farmID.
final_DF = pd.DataFrame() # Here we build a final dataframe to store all the value we got from each database (we did this above already ). 
for i in tqdm(range(len(farm.index))):
    tem_df = pd.DataFrame() # This is a temporary dataframe which we use to store values from each index. 
    #Get the column values for tem_df 
    placeID = farm.iloc[i,:]['placeID']
    workID = farm.iloc[i,:]['workID']
    farmID = farm.iloc[i,:]['farmID']
    long = farm.iloc[i,:]['long']
    lat = farm.iloc[i,:]['lat']
    altitude = farm.iloc[i,:]['altitude']
    conID = farm.iloc[i,:]['conID']
    soil = farm.iloc[i,:]['soil']
    area = farm.iloc[i,:]['area']
    tx01_max = farm.iloc[i,:]['tx01_max']
    tx01_min = farm.iloc[i,:]['tx01_min']
    wd01 = farm.iloc[i,:]['wd01']
    rh01 = farm.iloc[i,:]['rh01']
    ss01 = farm.iloc[i,:]['ss01']
    pp01 = farm.iloc[i,:]['pp01']
    #Get the hydro values and stored them in a list 
    hydro_lst = [tx01_max, tx01_min, wd01, rh01, ss01, pp01]
    #we loop through every station values in hydro_lst to get the hydro values from the list which we already got from the database.
    for index, hydro_code in enumerate(hydro_lst):
        tem_df = data_filledIn(tem_df, index, hydro_code, hydro_code)
    final_DF = pd.concat([final_DF, tem_df])
    
    final_DF['placeID'] = placeID
    final_DF['workID'] = workID
    final_DF['farmID'] = farmID
    final_DF['long'] = long
    final_DF['lat'] = lat
    final_DF['altitude'] = altitude
    final_DF['conID'] = conID
    final_DF['soil'] = soil
    final_DF['area'] = area

In [32]:
final_DF['conID'] = final_DF['conID'].map(lambda x:'鯉魚潭' if x =='Liyutan' else('寶山' if x=='Pao-Shan' else ('明德' if x=='Mingde' else('石岡壩' if x=='Ishioka' else('石門' if x=='Shimen' else x)))))
final_DF.rename(columns={'year':'年','month':'月','ten_days':'旬','placeID':'處代碼','workID':'工作站代碼','farmID':'坵塊編號','altitude':'高程(m)','conID':'競用區','long':'經度','lat':'緯度','tx01_max':'絕對最高氣溫(℃)','tx01_min':'絕對最低氣溫(℃)','wd01':'平均風速(m/s)','rh01':'相對溼度(%)','ss01':'平均日照時數','pp01':'累積降水量(mm)','soil':'土壤質地','area':'作物面積(m2)'},inplace=True)
final_DF[['年','月','旬','處代碼','工作站代碼','競用區','坵塊編號','經度','緯度','絕對最高氣溫(℃)','絕對最低氣溫(℃)','平均風速(m/s)','相對溼度(%)','平均日照時數','累積降水量(mm)','作物面積(m2)','高程(m)']]

,年,月,旬,處代碼,工作站代碼,坵塊編號,經度,緯度,海拔高度(m),絕對最高氣溫(℃),絕對最低氣溫(℃),平均風速(m/s),相對溼度(%),平均日照時數,累積降水量(mm),競用區,作物面積(平方公尺)
0,2022,1,1,7,716,BC202210110000A2,120.611569,24.14381,74,23.5,14.2,1.86,82.60,6.76,9.38,石岡壩,381.985765
1,2022,1,2,7,716,BC202210110000A2,120.611569,24.14381,74,24.2,11.0,1.89,80.52,6.24,2.90,石岡壩,381.985765
2,2022,1,3,7,716,BC202210110000A2,120.611569,24.14381,74,25.5,11.5,1.77,89.63,3.13,26.05,石岡壩,381.985765
3,2022,2,1,7,716,BC202210110000A2,120.611569,24.14381,74,23.2,12.4,1.90,88.37,4.68,15.22,石岡壩,381.985765
4,2022,2,2,7,716,BC202210110000A2,120.611569,24.14381,74,22.1,10.0,2.14,90.63,4.47,42.50,石岡壩,381.985765
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31,2022,11,2,7,716,BC202210110000A2,120.611569,24.14381,74,31.0,22.5,0.92,77.33,8.02,0.00,石岡壩,381.985765
32,2022,11,3,7,716,BC202210110000A2,120.611569,24.14381,74,33.2,20.4,1.06,78.40,4.22,14.92,石岡壩,381.985765
33,2022,12,1,7,716,BC202210110000A2,120.611569,24.14381,74,27.3,18.1,1.62,74.80,7.43,5.08,石岡壩,381.985765
34,2022,12,2,7,716,BC202210110000A2,120.611569,24.14381,74,24.2,10.5,1.76,70.69,5.11,9.44,石岡壩,381.985765


In [ ]:
final_DF.to_csv('C:\\Users\\tangkai\\Desktop\\final_DF.csv')

In [ ]:
#exe progress
final_DF = pd.read_csv("tem\\final_DF.csv")
exe_path = f'{os.getcwd()}\\Console1\\Console1.exe'
arg1 = 'argument1'
arg2 = 'argument2'
# Run the executable file with arguments
subprocess.run([exe_path, arg1, arg2])

In [ ]:
#change the original data columns to fix the input format
all_dff.rename(columns={"year":"年","month":"月","obshour":"時","stno":"測站編號","tx01_max":"絕對最高氣溫(℃)","tx01_min":"絕對最低氣溫(℃)","wd01":"累積日平均風速(m/s)","rh01":"相對溼度(%)","ss01":"累積日照時數","pp01":"累積降水量(mm)"},inplace=True)
farm.rename(columns={'處代碼':'placeID','工作站代碼':'workID','坵塊編號':'farmID','海拔高度(m)':'altitude','競用區':'conID','經度':'long','緯度':'lat','絕對最高氣溫(℃)':'tx01_max','絕對最低氣溫(℃)':'tx01_min','平均風速(m/s)':'wd01','相對溼度(%)':'rh01','平均日照時數':'ss01','累積降水量(mm)':'pp01','土壤質地':'soil','作物面積(平方公尺)':'area'},inplace=True)
farm['conID'] = farm['conID'].map(lambda x:'Liyutan' if x =='鯉魚潭' else('Pao-Shan' if x=='寶山' else ('Mingde' if x=='明德' else('Ishioka' if x=='石岡壩' else('Shimen' if x=='石門' else x)))))

In [ ]:
final_DF